[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ym001/Manteia/blob/master/notebook/notebook_Manteia_classification_cross_validation_run_in_colab.ipynb)


In [4]:
pip install manteia


In [6]:
#modifier->parametre du notebook->GPU
from Manteia.Classification import Classification 
from Manteia.Model import *
from Manteia.Dataset import Dataset
from Manteia.Preprocess import list_labels
from sklearn.model_selection import train_test_split,KFold

ds=Dataset('drugscom')

model = Model(model_name ='bert',early_stopping=True)
model.load_type()
model.load_tokenizer()
list_label=list_labels(ds.labels_train)
print(list_label)
model.num_labels=len(list_label)
model.load_class()
model.save('model_init')
train_ids,train_masks           = encode_text(ds.documents_train[:500],model.tokenizer,model.MAX_SEQ_LEN)
train_labels                    = encode_label(ds.labels_train[:500],list_label)
train_ids,train_masks,train_labels = np.array(train_ids),np.array(train_masks),np.array(train_labels)

#validation croisée
nb_pass=4
def coss_validation_idx(nb_pass,nb_docs):
  docs_idx = [idx for idx in range(nb_docs)]
  train_idx, test_idx = [], []
  for pli in range(nb_pass):
    test_pli_idx = list(np.random.choice(docs_idx,int(len(docs_idx)/nb_pass) , replace=False))
    train_pli_idx  = [idx for idx in docs_idx if idx not in test_pli_idx]
    train_idx.append(train_pli_idx)
    test_idx.append(test_pli_idx)
  return train_idx, test_idx
pli=0
acc = []
tr_idx,te_idx=coss_validation_idx(nb_pass,len(train_ids))
for train_idx, test_idx in zip(tr_idx,te_idx):
  validation_idx = list(np.random.choice(train_idx,int(0.1*len(train_idx)) , replace=False))
  train_idx      = [idx for idx in train_idx if idx not in validation_idx]
  dt_train       = Create_DataLoader(train_ids[train_idx],train_masks[train_idx],train_labels[train_idx])
  dt_validation  = Create_DataLoader(train_ids[validation_idx],train_masks[validation_idx],train_labels[validation_idx])
  dt_test        = Create_DataLoader(train_ids[test_idx],train_masks[test_idx])
  model.load('model_init')

  model.configuration(dt_train)
  model.fit(dt_train,dt_validation)

  predictions=model.predict(dt_test,p_type='class')
  acc.append(accuracy(train_labels[test_idx],predictions))
  print('Passe cross validation : {} , Accuracy : {}'.format(pli,acc[-1]))
  pli+=1

print('')
print('Cross validation accuracy : {} : {}'.format(np.mean(np.array(acc)),acc))


Loading bert tokenizer...
['1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0']
Loading bert class...


Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


type compatible : bert-base-uncased
Loading bert class...
There are 1 GPU(s) available.
We will use the GPU: Tesla K80
There are 1 GPU(s) available.
We will use the GPU: Tesla K80

======== Epoch 1 / 20 ========


  Average training loss: 2.11


Validation : Accuracy : 0.27
Validation accuracy increased (0.000000 --> 0.270270).  Saving model ...

======== Epoch 2 / 20 ========


  Average training loss: 1.98


Validation : Accuracy : 0.30
Save model : 0 out of 2
Validation accuracy increased (0.270270 --> 0.297297).  Saving model ...

======== Epoch 3 / 20 ========


  Average training loss: 1.87


Validation : Accuracy : 0.30
Save model : 0 out of 2
Validation accuracy increased (0.297297 --> 0.297297).  Saving model ...

======== Epoch 4 / 20 ========


  Average training loss: 1.74


Validation : Accuracy : 0.35
Save model : 0 out of 2
Validation accuracy increased (0.297297 --> 0.351351).  Saving model ...

======== Epoch 5 / 20 ========


  Average training loss: 1.56


Validation